In [29]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from amb import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ALLCools.plot import *
age_palette = get_age_palette()
import glob
import pathlib
import re
import cooler
from collections import defaultdict

In [7]:
group_name = 'DG_Glut'

In [8]:
ct = group_name

In [9]:
gene = 'Il33'

In [10]:
chrom_size_path = '/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
gene_meta_path = '/data/metadata/gencode.vM22.basic.annotation.gene.flat.tsv.gz'
dmr_zarr_path = f"{ct}.AllDMR.mcds"
cool_path = '/data/female-amb/AMB.CoolFiles/CellType.Age.Raw.5kb.mcool'
leg = [f"{ct}.{age}" for age in ['8wk','9mo','18mo']]
age_order = ['2mo','9mo','18mo']

In [13]:
mouse_chrs = ['chr' + str(x) for x in range(1,20)] + ['chrX']
mouse_size = pd.read_csv(chrom_size_path, sep="\t", index_col=0, names=['length']).loc[mouse_chrs]

mouse_genes = pd.read_csv(gene_meta_path,sep="\t")[['chrom', 'start', 'end', 'gene_id','gene_name']]
mouse_genes.columns = ['chrom', 'start', 'end', 'geneID','gene_name']
mouse_genes['geneID'] = [re.sub("\.[0-9]+$", "", x) for x in mouse_genes['geneID']]
mouse_genes.index = mouse_genes['geneID']
mouse_genes = mouse_genes[~mouse_genes['chrom'].isin(['chrY', 'chrM'])]

In [15]:
atac_rpm_path = f"celltype_age_RPM_files/{ct}_RPM.txt"
atac_rpm =pd.read_csv(atac_rpm_path, sep = '\t')

In [16]:
peak_bed = pd.DataFrame({'chrom':[_id.split(':')[0] for _id in atac_rpm.index],
                         'start':[int(_id.split(':')[1].split('-')[0]) for _id in atac_rpm.index],
                         'end':[int(_id.split(':')[1].split('-')[-1]) for _id in atac_rpm.index]}, index = atac_rpm.index)
peak_bed.head()

,chrom,start,end
chr1:3012480-3012981,chr1,3012480,3012981
chr1:3060761-3061262,chr1,3060761,3061262
chr1:3113502-3114003,chr1,3113502,3114003
chr1:3112919-3113420,chr1,3112919,3113420
chr1:3120282-3120783,chr1,3120282,3120783


In [17]:
atac_rpm =pd.read_csv(atac_rpm_path, sep = '\t')
atac_rpm.columns = [_.split(':')[-1] for _ in atac_rpm.columns] 
dmr_activity_dict = atac_rpm[age_order].T
dmr_activity_dict.index = leg
dmr_activity_dict = dmr_activity_dict.to_dict()

In [20]:
dmr_activity_dict

{'chr1:3012480-3012981': {'DG_Glut.8wk': 2.53519121093935,
  'DG_Glut.9mo': 2.24078241745563,
  'DG_Glut.18mo': 2.09644442683397},
 'chr1:3060761-3061262': {'DG_Glut.8wk': 1.62688603433526,
  'DG_Glut.9mo': 1.17678673936848,
  'DG_Glut.18mo': 1.15076569081647},
 'chr1:3113502-3114003': {'DG_Glut.8wk': 5.27433606466708,
  'DG_Glut.9mo': 4.42140960222789,
  'DG_Glut.18mo': 4.15490978796443},
 'chr1:3112919-3113420': {'DG_Glut.8wk': 2.16522878913716,
  'DG_Glut.9mo': 1.65426889879275,
  'DG_Glut.18mo': 1.64829145153251},
 'chr1:3120282-3120783': {'DG_Glut.8wk': 2.3596962159819,
  'DG_Glut.9mo': 2.2295033113275,
  'DG_Glut.18mo': 2.11163605311538},
 'chr1:3119730-3120231': {'DG_Glut.8wk': 1.67906022202531,
  'DG_Glut.9mo': 1.48132260482805,
  'DG_Glut.18mo': 1.48877937557775},
 'chr1:3119075-3119576': {'DG_Glut.8wk': 1.02688287589966,
  'DG_Glut.9mo': 0.947444914763118,
  'DG_Glut.18mo': 0.884912230891877},
 'chr1:3204691-3205192': {'DG_Glut.8wk': 1.14546057519523,
  'DG_Glut.9mo': 1.07527

In [26]:
group = f"{ct}.8wk"

In [35]:
gene_id = mouse_genes[mouse_genes['gene_name'] == 'Il33'].index[0]

In [36]:
ct_age_cool = cooler.Cooler(f"/data/female-amb/loop/{group}/{group}/{group}.Q.cool")

In [37]:
max_distance = 5000000
ABC_score = defaultdict(dict)
total_ABC = dict()

gene_coords = mouse_genes.loc[gene_id].to_dict()
chrom = gene_coords['chrom']
gene_start = int(gene_coords['start']) - 2000
gene_end = int(gene_coords['end']) + 2000

start = int(gene_coords['start']) - max_distance
end = int(gene_coords['start']) + max_distance
start = 1 if start < 0 else start
end = mouse_size.loc[chrom, 'length'] if end > mouse_size.loc[chrom, 'length'] else end


In [44]:
ct_age_dmr = peak_bed
gene_dmr = ct_age_dmr[(ct_age_dmr['chrom'] == gene_coords['chrom']) & (ct_age_dmr['start'] > start)
                                   & (ct_age_dmr['end'] < end)]
gene_dmr = gene_dmr[gene_dmr['end'] - gene_dmr['start']>=10]

In [45]:
# calculate interactions of each DMR to target gene
dmr_contacts = defaultdict(dict)
contacts = ct_age_cool.matrix(balance=False, as_pixels=True, join=True).fetch(f'{chrom}:{start}-{end}')

gene_contacts_upper = contacts[(contacts['start1'] >= gene_start) & (contacts['start1'] <= gene_end)]
gene_contacts_down = contacts[(contacts['start2'] >= gene_start) & (contacts['start2'] <= gene_end)]
gene_contacts_upper = gene_contacts_upper[(gene_contacts_upper['start2'] >= gene_start) & (gene_contacts_upper['start2'] <= end)]
gene_contacts_down = gene_contacts_down[(gene_contacts_down['start1'] >= start) & (gene_contacts_down['start1'] <= gene_start)]

In [62]:
for z, row in gene_contacts_upper.iterrows():
        z_bin_dmrs = gene_dmr[(gene_dmr['start'] >= row.start2) & (gene_dmr['end'] <= row.end2)]         
        for dmr in z_bin_dmrs.index:
            dmr_contacts[dmr] = row['count']

for y, row in gene_contacts_down.iterrows():
    y_bin_dmrs = gene_dmr[(gene_dmr['start'] >= row.start1) & (gene_dmr['end'] <= row.end1)]         
    for dmr in y_bin_dmrs.index:
        dmr_contacts[dmr] = row['count']

total_ABC =[]
for dmr in dmr_contacts:
    total_ABC.append(dmr_activity_dict[dmr][group] * dmr_contacts[dmr])
total_ABC = sum(total_ABC)

In [63]:
for dmr in dmr_contacts:
    EG = f'{dmr}-{gene_id}'
    try:
        activity = dmr_activity_dict[dmr][group]
        contact = dmr_contacts[dmr]
        ABC_score[EG] = activity, contact, (activity *  contact/ total_ABC)
    except:
        ABC_score[EG][celltype] = np.nan

In [64]:
ABC_score_df = pd.DataFrame.from_dict(ABC_score, orient='index')

In [66]:
ABC_score_df[2].describe()

count    1123.000000
mean        0.000890
std         0.004556
min         0.000002
25%         0.000044
50%         0.000093
75%         0.000298
max         0.078910
Name: 2, dtype: float64